# Projeto 1 - Ciência dos Dados

Nome: Guilherme Aranha

Nome: André Costa

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import emoji

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [2]:
import os

filename = 'airpods.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo airpods.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
train = pd.read_excel(filename)
train.head(5)

,Tweet,Classe
0,@macmagazine @eduardomarques só é válido para ...,1
1,alfândega dos eua apreende 2.000 gemas oneplus...,1
2,você tem airpods ? onde vc comprou nunca achei...,2
3,"pensando seriamente em compra airpods, e to de...",0
4,perdi meus airpods dnv 🤣🤣🤣😂😂,1


In [4]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Tweet,Classe
0,as vezes eu não entendo meu pai kkk eu falei q...,2
1,"ganhei um airpods original, carregador origina...",1
2,@rodrigosoutom @dorafigueiredo não tem outra f...,2
3,@claraaamoraiss “sabes que os meus airpods caí...,1
4,esses airpods pro são uns guerreiros msm pq o ...,2


___
## Classificador automático de sentimento


Nosso produto é os **AIRPODS**, os fones true wireless da Apple. Por ser um objeto top de linha, escolhemos classificar as opinioes apenas como positivas, negativas e neutras, descartando a ideia de criar subcategorias como "um pouco positivo" e "muito positivo".

As três categorias que usamos são:
* **Positiva:** todos os tweets que tem um vies positivo do produto
* **Neutra:** todos os tweets que informam algo sem nenhum vies
* **Negativa:** todos os tweets que tem um vies negativo do produto
    
Como estamos usando um classificador Naive Bayes, usamos na nossa base de dados apenas a frequência com que as palavras aparecem nos tweets, descartando qualquer outro método de analize como a ordem delas.

Para facilitar a interpretação dos texto (que são frequentemente muito mal escritos nessa rede), limpamos as strings:
* Retiramos alguns símbolos, mas não emojis;
* retiramos links;
* separamos emojis como se cada um fosse uma palavra individual, mesmo que digitados juntos nos tweets

(Não removemos @ porque ele simboliza que um usuário esta sendo marcado).

Para obter uma base de dados de qualidade, buscamos equilibrar a quantidade de palavras vindas de tweets positivos, negativos e neutros que seriam usados para treinar o algoritmo (a quantidade de tweets ser desequilibrada não afeta a eficiência do algoritmo, precisamos apenas de uma quantidade de palavras parecida uma vez que analisamos apenas a frequência delas).

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [5]:
def cleanup(text):
    '''
    Função que recebe uma string e retorna essa mesma string com algumas correções/filtrações.
    '''
    
    text_cleaned = re.sub(re.compile('[!-.:?;\n]'), '', text)               #substitui os caracteres ! - . : ? ; \n por uma string vazia
    text_cleaned = re.sub(re.compile(r'(http\S*\s)'), '', text_cleaned)     #substitui links básicos por uma string vazia
    
    text_list = ''                                                          #checa caractere por caracter para adicionar um espaço antes e um depois se ele for um emoji
    for x in text_cleaned:
        if x in emoji.UNICODE_EMOJI:
            text_list += f' {x} '
        else:
            text_list += f'{x}'
    text_cleaned = text_list
    
    text_cleaned = re.sub(re.compile(r'\s+'), ' ', text_cleaned)            #substitui sequencias de espaços por um único espaço

    return text_cleaned

classes = [('neg', 0), ('neu', 1), ('pos', 2)]      #lista dos nomes e dos respectivos numeros das classificações usadas
nomes = {'neg': 'Negativo', 'neu': 'Neutro', 'pos': 'Positivo'} #nomes para usar print de forma mais elegante
string = {}                                         #dicionário que vai conter uma string para cada classe composta por todos os tweets dessa classe
for classe, n in classes:                           #uma vez para cada classe
    frame = train.loc[train['Classe'] == n, :]      #seleciona só os tweets da classe
    string[classe] = ''                             #cria uma string para compreender os tweets
    for tweet in frame['Tweet']:                    #adiciona cada tweet na string 
        string[classe] += ' ' + tweet
    
    string[classe] = cleanup(string[classe].lower()) #faz a limpeza da string

string['pos'][:1000]

' você tem airpods onde vc comprou nunca achei aq em araxa — comprei no site da apple só queria um airpods 🥴 ganhei um fucking airpods pro de aniversario caralho queria muito @lmwtff manoooo eu tbm to querendo e um airpods tbm huahuah como segurar essa vontade de comprar kkk para as pessoas comprarem airpods lógico eu to fazendo de tudo pra não pagar 1600 reais em um airpods pro mas tá difícil viu 🤦 🏻 \u200d ♂ ️ manda um airpods aq e eu falo se esperava ou n hoje reclamei com a loja porque os meus airpods nunca mais chegavam e fizeram me o reembolso and guess what chegaram mesmo agora já to juntando dinheiro pro airpods studio veyr o lançamento de hoje vai ser pra mim appleevent @llbggp sempre de airpods meus airpods fazem parte do meu corpo qualquer lugar q eu to eles têm q estar tbm 🤧 tenho que comprar meu airpods porra feliz que meu airpods tá funcionando nesse celular gratidão tirei o powerbeats pro da gaveta pra usar na caminhada/corrida e por mais que eu prefira os airpods pro ô 

In [6]:
t = 0             #print da quantidade de palavras em cada classe na base de treino
for classe, n in classes:
    t += len(pd.Series(string[classe].split()).value_counts())
for classe, n in classes:
    print(f'palavras na classe {classe}:',len(pd.Series(string[classe].split()).value_counts()), f'({(len(pd.Series(string[classe].split()).value_counts())/t)*100:.3g}%)')

palavras na classe neg: 791 (26.9%)
palavras na classe neu: 1226 (41.7%)
palavras na classe pos: 921 (31.3%)


In [7]:
alpha = 1       #constante alpha para ser utilizada caso precisemos usar uma palavra que não consta na base de dados
V = 10**6       #constante V para representar a quantidade de palavras póssiveis de usarmos em tweets

valores = {}    #dicionário que vai conter um DataFrame para cada classe informando o peso de cada palavra na base de dados da classe
for classe in string: #conversão das strings para o DataFrame dos pesos
    valores[classe] = (pd.Series(string[classe].split()).value_counts() + alpha) / (len(string[classe].split()) + alpha*V)

valores['pos'].head()

airpods    0.000171
de         0.000068
que        0.000065
um         0.000063
o          0.000055
dtype: float64

In [8]:
def classifica(tweet):
    '''
    Função que usa a base de dados da variável valores para classificar tweets na forma de uma única string em positivo, negativo ou neutro.
    '''
    tweet_clean = cleanup(tweet)
    frase_list = tweet_clean.split()                        #lista com cada palavra da frase
    resultados = []                                         #lista que vai compreender uma tupla (classe, peso do tweet na respectiva classe) para cada classe
    for classe, n in classes:                               #uma vez para cada classe
        value = 0                                           #variavel que copreende a combinação dos pesos de cada palavra do tweet na classe da vez
        for palavra in frase_list:                          #uma vez para cada palavra no tweet
            if palavra in valores[classe]:                  #se a palavra constar na base de dados...
                value += np.log(valores[classe][palavra])   #adiciona o peso dela na variavel value
                #print(classe, palavra, np.log(valores[classe][palavra]), '(tem)')
            else:
                value += np.log(alpha / (len(string[classe].split()) + alpha*V)) #adiciona o peso de uma palavra genérica na variavel value
                #print(classe, palavra, np.log(alpha / (len(string[classe].split()) + alpha*V)), '(NAO tem)')
        resultados += [(value, n)]                          #adiciona a tupla (classe, peso do tweet na respectiva classe) na lista de resultados
    classificacao = (-10000, 0)                                  #variavel que vai compreender a tupla com o maior peso
    for value, n in resultados:                             #uma vez par cada tupla na lista de resultados
        if classificacao[0] < value:                        #se o peso dessa tupla for maior do que o da variavel classificação...
            classificacao = (value, n)                      #a variavel classificação assumi o valor dessa tupla
    
    return classificacao[1]                                 #retorna apenas o número da classifição alcançada



print(classifica('ganhei novos airpods'))

2


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [9]:
perf = {} #dicionário que vai compreender os fesultados da verificação de performance
for classe, n in classes:
    perf[classe] = {}
    for classe_B, n_B in classes:
        perf[classe][classe_B] = 0
    frame = test.loc[train['Classe'] == n, :] #seleciona os tweets da planilha de teste da respectiva classe
    for tweet in frame['Tweet']:
        perf[classe][classes[classifica(tweet)][0]] += 1
        
perf

{'neg': {'neg': 2, 'neu': 27, 'pos': 17},
 'neu': {'neg': 6, 'neu': 72, 'pos': 36},
 'pos': {'neg': 2, 'neu': 57, 'pos': 33}}

In [10]:
perf_clean = {'total':{'verdadeiro':0, 'falso':0}} #variável perf pronta para expor os dados no print
for classe, n in classes:
    perf_clean[classe] = {'verdadeiro':0, 'falso':0}
    for classe_B, n_B in classes:
        if classe == classe_B:
            perf_clean[classe]['verdadeiro'] += perf[classe_B][classe]
            perf_clean['total']['verdadeiro'] += perf[classe_B][classe]
        else:
            perf_clean[classe]['falso'] += perf[classe_B][classe]
            perf_clean['total']['falso'] += perf[classe_B][classe]

perf_clean

{'total': {'verdadeiro': 107, 'falso': 145},
 'neg': {'verdadeiro': 2, 'falso': 8},
 'neu': {'verdadeiro': 72, 'falso': 84},
 'pos': {'verdadeiro': 33, 'falso': 53}}

In [11]:
for classe, n in classes: #print dos resultados da verificação de performance
    print(f'{nomes[classe]}:\n Verdadeiros ' + nomes[classe] + f': {perf_clean[classe]["verdadeiro"]} ({(perf_clean[classe]["verdadeiro"] / sum(perf_clean["total"].values()))*100:.3g}%)\n Falsos ' + nomes[classe] + f': {perf_clean[classe]["falso"]} ({(perf_clean[classe]["falso"] / sum(perf_clean["total"].values()))*100:.3g}%)')
print(f'Total:\n Verdadeiros total: {perf_clean["total"]["verdadeiro"]} ({(perf_clean["total"]["verdadeiro"] / sum(perf_clean["total"].values()))*100:.3g}%)\n Falsos total: {perf_clean["total"]["falso"]} ({(perf_clean["total"]["falso"] / sum(perf_clean["total"].values()))*100:.3g}%)')

Negativo:
 Verdadeiros Negativo: 2 (0.794%)
 Falsos Negativo: 8 (3.17%)
Neutro:
 Verdadeiros Neutro: 72 (28.6%)
 Falsos Neutro: 84 (33.3%)
Positivo:
 Verdadeiros Positivo: 33 (13.1%)
 Falsos Positivo: 53 (21%)
Total:
 Verdadeiros total: 107 (42.5%)
 Falsos total: 145 (57.5%)


___
### Concluindo

Como podemos ver na cessão anterior do notebook, nosso classificador conseguiu interpretar corretamente 42.5% dos tweets da planilha Teste. Avaliamos que uma eficiência de 42.5% não é muito boa. Diversas fontes falem que o Naive Bayes consegue uma eficiencia de até 92%. Supomos que não conseguimos alcançar essa taxa tão alta porque nossa base de dados não foi tão grande.

Ao fazer algumas experiências com nosso classificador concluimos que a dupla negação é interpretada pelo algoritmo como "algo que nega muito" e não como uma negação cancelando a outra. Dessa forma, duplas negações tendem a ter uma taxa de acerto muito baixa no classificador.
O sarcasmo é interpretrado como a exata mesma frase, mas sem sarcasmo, logo, tendo o sentido oposto. Isso acontece porque o sarcasmo da forma que entendemos em nossas comunições sociais não é interpretavel baseado apenas na frequência de palavras na frase sarcástica.

Como podemos ver, nossa base de tweets de treinamento é baixa e desequilibrada (aprox. 31% positivo, 42% neutro e 27% negativo). Assim, caso ocorra um plano de expansão do nosso projeto, seria possível aumentar a nossa base de tweets e buscar equilibrar a porcentagem de palavras de tweets positivos, neutros e negativos, e com isso melhorar nossa taxa de acerto.
Com uma maior eficiência, poderiamos expandir esse projeto para ver a avaliação do público na internet da própria empresa Apple e também todos os seus produtos, para ajudar no marketing em geral.

Analisando e compreendendo melhor o funcionamento do algoritmo Naive Bayes, fica aparente o motivo pelo qual não podemos alimentar nossa base de treinamento automaticamente usando o próprio classificador: sabendo que quando usamos nosso algoritmo para classificar uma quantidade X de dados, ele obtem duas frações de X, uma corretamente avaliada e outra errada. Se adicionarmos X inteiro na base de treinamento, cada fração de X terá um efeito difrente. A fração corretamente avaliada apenas reproduz o que já estava na base de dados, "ele aprende mais a mesma coisa, logo não aprendendo nada". A fração errada acaba sendo o problema, ela polui a base de treinamento com dados erroneos, assim diminuindo a eficiência do algoritmo

Pensando além do projeto, uma possível aplicação do Naive Bayes é na construção de um sistema de reconhecimento de doenças a partir da combinação da presença e/ou ausência de sintomas diversos na vítima. Não seria um sistema usado por profissionais ou para substituir profissionais da área médica, uma vez que a eficiência desse algoritmo é boa, mas não melhor do que a dos especialistas. Seria uma ferramenta de facil acesso, como uma primeira ajuda para alguém que está indeciso se deve ou não ir ao hospital atrás de um diagnóstico profissional.

Mesmo que o Naive Bayes tenha uma grande eficiência frente a necessidade de poder computacinal e complexidade muito baixas, ainda há muito que poderia ser implementado para melhorar ainda mais sua eficiência, mesmo que a custo da simplicidade. Uma das mais simples implementações que podem ser usadas no Naive Bayes é manualmente retirar da base de dados palavras que aparecem com muita frequência mas que claramente não aprensentam nenhum valor para as classificações ddo algoritmo. Poderiamos facilmente retirar artigos da base de treinamento, por exemplo. FONTE >>> https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**